In [49]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn import metrics
import warnings
warnings.filterwarnings("ignore")

In [61]:
def transform_percentage(data_origin, col, piece):
    position_0 = min(data_origin[col])-2
    data = data_origin
    trans = np.zeros(len(data_origin[col]))
    gap = int(len(data_origin[col])/piece)
    #for i in range(len(data_origin[col])):
        #print(i)
    for k in range(piece):
        if k == piece-1:
            #print('position_0',position_0)
            position_1 = max(data[col])
            #print('position_1',position_1)
            range_data = data[data[col].between(position_0+1, position_1)]
            #print(range_data)
        else:
            position = min(gap*(k+1), len(data[col]))
            position_1 = sorted(data[col])[position]
            #print(position)
            #print('position_0+1',position_0+1)
            #print('position_1',position_1)
            range_data = data[data[col].between(position_0+1, position_1)]
            #print(range_data)
            position_0 = position_1
        for i in range(range_data.shape[0]):
            trans[range_data.index[i]]=k
        #print(data[col])
    data[col] = trans
    #return data

In [4]:
def categorical_transform(data, col):
    count = pd.DataFrame(data[col].value_counts())
    class_mapping = {}
    for i in range(count.shape[0]):
        add = {count.index[i]:i}
        class_mapping.update(add)
    data[col] = data[col].map(class_mapping)

In [17]:
def transform_numerical(data_origin, col, numerical):
    data = data_origin
    data[col] = data[col].map(lambda x:x*numerical)
    data[col] = data[col].astype(int)
    #data[col] = data[col].map(lambda x:x*numerical)
    #return data

In [16]:
def train_test_ch(data, test, col, binary_class):
    #result = pd.DataFrame()
    feature = data.loc[:, [col]]
    feature_count = feature[col].value_counts()
    feature_pd = pd.DataFrame(feature_count)
    feature_reset = feature_pd.reset_index()
    feature_sort = feature_reset.sort_values(by='index')
    
    feature_1 = data[data[binary_class] == 1]
    feature_1 = feature_1.loc[:, [col]]
    feature_1_count = feature_1[col].value_counts()
    feature_1_pd = pd.DataFrame(feature_1_count)
    feature_1_reset = feature_1_pd.reset_index()
    feature_1_sort = feature_1_reset.sort_values(by='index')
    
    feature_merge = pd.merge(feature_sort, feature_1_sort, on='index', how='left')
    feature_fill = feature_merge.fillna(0)
    feature_fill = feature_fill.rename(columns={'index':col})
    #print(feature_fill)
    #del feature_fill[col+'_x']
    #del feature_fill[col+'_y']
    #feature_fill[col+'_fraud_ratio'] = feature_fill[col+'_y']/feature_fill[col+'_x']
    result_0 = pd.DataFrame(feature_fill.iloc[:,2]/feature_fill.iloc[:,1], columns = [col+'_fraud_ratio'])
    result_1 = pd.concat([feature_fill[[col]], result_0], axis = 1)
    describe_pd = pd.DataFrame(result_0[col+'_fraud_ratio'].describe())
    fill = describe_pd.loc['mean', col+'_fraud_ratio']
    #result = pd.concat([result, feature_fill[[col]], result_0], axis = 1)
    train = pd.merge(data, result_1, on = col, how = 'left')
    test = pd.merge(test, result_1, on = col, how = 'left')
    test = test.fillna(fill)
    #print(test)
    return train, test

In [26]:
def final_data_train_test(train, test, binary_class):
    final_train = train
    final_test = test
    final_train['Ratio'] = 1
    final_test['Ratio'] = 1
    for i in range(train.shape[1]-2):
        final_train = train_test_ch(final_train, final_test, train.columns[i], binary_class)[0]
        final_test = train_test_ch(final_train, final_test, train.columns[i], binary_class)[1]
        #del new_feature[data.columns[i]+'_x']
        #del new_feature[data.columns[i]+'_y']
        #final = pd.merge(final, new_feature, on = data.columns[i], how = 'left')
        #print(final_train)
        final_train['Ratio'] = final_train['Ratio']*final_train[train.columns[i]+'_fraud_ratio']
        final_test['Ratio'] = final_test['Ratio']*final_test[test.columns[i]+'_fraud_ratio']
    return final_train, final_test

In [38]:
def result(data, threshold, binary_class):
    predict = np.array(data['Ratio'])
    for i in range(0,data.shape[0]):
        if predict[i] >= threshold:
            predict[i]=1
        else:
            predict[i]=0
    y_true = data[binary_class]
    y_pred = predict
    auc = round(roc_auc_score(y_true, data['Ratio']),4)
    f1_score = round(metrics.f1_score(y_true, y_pred),4)
    accuracy = round(metrics.accuracy_score(y_true, y_pred),4)
    precision = round(metrics.precision_score(y_true, y_pred),4)
    recall = round(metrics.recall_score(y_true, y_pred),4)
    print('Accuracy:', accuracy)
    print('Precision:', precision)
    print('Recall:', recall)
    print('F1-score:', f1_score)
    print('AUC:', auc)

In [73]:
def data_analysis_train_test(data, binary_class):
    x_train = data_d[data_d['locdt'] <= 70]
    x_test = data_d[data_d['locdt'] > 70]
    x_train = x_train.drop('locdt', axis=1)
    x_test = x_test.drop('locdt', axis=1)
    y_train = train[binary_class]
    y_test = test[binary_class]
    #x_train = train.drop([binary_class], axis = 1)
    #x_test = test.drop([binary_class], axis = 1)
    final_train = final_data_train_test(x_train, x_test, binary_class)[0]
    final_test = final_data_train_test(x_train, x_test, binary_class)[1]
    describe_train_pd = pd.DataFrame(final_train['Ratio'].describe())
    describe_test_pd = pd.DataFrame(final_test['Ratio'].describe())
    threshold_train = describe_train_pd.loc['mean','Ratio']
    threshold_test = describe_test_pd.loc['mean','Ratio']
    #final_data_train = final_data(x_train, binary_class)
    #describe_pd = pd.DataFrame(final_data_train['Ratio'].describe())
    #threshold = describe_pd.loc['75%','Ratio']
    print('Train data result','\n')
    result(final_train, threshold_train, binary_class)
    print('\n')
    #final_data_test = final_data(x_test, binary_class)
    print('Test data result','\n')
    result(final_test, threshold_test, binary_class)

In [65]:
data = pd.read_csv('train.csv')

In [66]:
data_d = data.drop(['acqic', 'bacno', 'cano', 'txkey', 'flbmk', 'flg_3dsmk'], axis = 1)

In [67]:
categorical_transform(data_d, 'ecfg')
categorical_transform(data_d, 'insfg')
categorical_transform(data_d, 'ovrlt')

In [27]:
data_d.head()

,conam,contp,csmcu,ecfg,etymd,fraud_ind,hcefg,insfg,iterm,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,stscd
0,5,5,0,0,0,0,5,0,0,33,17,45,59,0,0,102,0
1,4,5,0,0,2,0,0,0,0,9,10,45,0,0,5817,102,0
2,5,5,0,0,0,0,5,0,0,6,15,45,59,0,0,102,0
3,10,5,62,0,5,0,5,0,0,5,17,24,50,0,3281,102,0
4,7,5,62,0,4,0,5,0,0,6,18,26,93,0,5817,102,0


In [68]:
transform_numerical(data_d, 'mcc', 1/10)

In [69]:
transform_numerical(data_d, 'mchno', 1/1000)

In [70]:
transform_numerical(data_d, 'loctm', 1/10000)

In [24]:
#transform_numerical(data_d, 'conam', 1/100)

In [48]:
#transform_numerical(data_d, 'conam', 100)

In [71]:
transform_percentage(data_d, 'conam', 10)

In [72]:
fraud = data_d.loc[:,['fraud_ind']]

In [74]:
data_d = data_d.drop(['fraud_ind'], axis = 1)
data_d = pd.concat([data_d, fraud], axis = 1)

In [75]:
data_d.head()

,conam,contp,csmcu,ecfg,etymd,hcefg,insfg,iterm,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,stscd,fraud_ind
0,3.0,5,0,0,0,5,0,0,33,17,45,59,0,0,102,0,0
1,2.0,5,0,0,2,0,0,0,9,10,45,0,0,5817,102,0,0
2,3.0,5,0,0,0,5,0,0,6,15,45,59,0,0,102,0,0
3,8.0,5,62,0,5,5,0,0,5,17,24,50,0,3281,102,0,0
4,6.0,5,62,0,4,5,0,0,6,18,26,93,0,5817,102,0,0


In [76]:
data_analysis_train_test(data_d, 'fraud_ind')

Train data result 

Accuracy: 0.9867
Precision: 0.5733
Recall: 0.3318
F1-score: 0.4204
AUC: 0.9647


Test data result 

Accuracy: 0.9894
Precision: 0.4035
Recall: 0.256
F1-score: 0.3132
AUC: 0.9186


old code

In [47]:
data_d.shape

(1184411, 18)

In [48]:
result = pd.DataFrame()
#for col in ['contp', 'csmcu', 'etymd', 'mcc', 'hcefg', 'stocn', 'scity']:
data_m = data_d
for col in ['stocn', 'csmcu', 'etymd', 'mcc', 'mchno', 'loctm', 'conam']:
    data_d_count = data_d[col].value_counts()
    data_d_pd = pd.DataFrame(data_d_count)
    data_d_reset = data_d_pd.reset_index()
    data_d_sort = data_d_reset.sort_values(by='index')
    
    data_d_1 = data_d[data_d['fraud_ind'] == 1]
    data_d_1 = data_d_1.loc[:,[col]]
    data_d_1_count = data_d_1[col].value_counts()
    data_d_1_pd = pd.DataFrame(data_d_1_count)
    data_d_1_reset = data_d_1_pd.reset_index()
    data_d_1_sort = data_d_1_reset.sort_values(by='index')

    data_d_merge = pd.merge(data_d_sort,data_d_1_sort,on = 'index',how = 'left')
    data_d_fill = data_d_merge.fillna(0)
    
    data_d_fill = data_d_fill.rename(columns = {'index': col})
    
    result_0 = pd.DataFrame(data_d_fill.iloc[:,2]/data_d_fill.iloc[:,1], columns = [col+'_pro'])
    result_1 = pd.concat([data_d_fill[[col]], result_0], axis = 1)
    
    result = pd.concat([result, data_d_fill[[col]], result_0], axis = 1)
    
    data_m = pd.merge(data_m, result_1, on = col, how = 'left')
    x_test = pd.merge(x_test, result_1, on = col, how = 'left')


In [49]:
data_m['pro'] = data_m['stocn_pro']*data_m['csmcu_pro']*data_m['etymd_pro']*data_m['mcc_pro']*data_m['mchno_pro']*data_m['loctm_pro']*data_m['conam_pro']
x_test['pro'] = x_test['stocn_pro']*x_test['csmcu_pro']*x_test['etymd_pro']*x_test['mcc_pro']*x_test['mchno_pro']*x_test['loctm_pro']*x_test['conam_pro']

In [50]:
data_m['pro'].describe()

count    1.184411e+06
mean     2.725680e-10
std      5.407398e-09
min      0.000000e+00
25%      5.484526e-19
50%      6.191372e-18
75%      3.720928e-16
max      3.240382e-07
Name: pro, dtype: float64

In [51]:
x_test['pro'].describe()

count    3.373560e+05
mean     2.002471e-10
std      4.842952e-09
min      0.000000e+00
25%      4.998366e-19
50%      4.048939e-18
75%      1.689769e-16
max      3.141390e-07
Name: pro, dtype: float64

In [13]:
x_test['pro'].describe()

count    3.373560e+05
mean     2.002471e-10
std      4.842952e-09
min      0.000000e+00
25%      4.998366e-19
50%      4.048939e-18
75%      1.689769e-16
max      3.141390e-07
Name: pro, dtype: float64

In [65]:
pro1 = np.array(data_m['fraud_ind_1'])
pro2 = np.array(data_m['pro'])
#data_m['stocn_pro']*data_m['csmcu_pro']*data_m['etymd_pro']*data_m['mcc_pro']
#*data_m['mchno_pro']*data_m['loctm_pro']*data_m['conam_pro']
pro3 = np.array(data_m['stocn_pro'])
pro4 = np.array(data_m['csmcu_pro'])
pro5 = np.array(data_m['etymd_pro'])
pro6 = np.array(data_m['mcc_pro'])
pro7 = np.array(data_m['mchno_pro'])
pro8 = np.array(data_m['loctm_pro'])
pro9 = np.array(data_m['conam_pro'])
for i in range(len(pro1)):
    if pro2[i] >= 2.725680e-10:
        pro1[i]=1 
    else:
        pro1[i]=0
#for i in range(len(pro1)):
#    if pro3[i] or pro4[i] or pro5[i] or pro6[i] or pro7[i] or pro8[i] or pro9[i] == 0:
#        pro1[i] =0
#    elif pro3[i] or pro4[i] or pro5[i] or pro6[i] or pro7[i] or pro8[i] or pro9[i] == 1:
#        pro1[i] = 1
        
        

In [66]:
x = pd.DataFrame(pro1,columns=['result'])
print(x['result'].value_counts())

0.0    1158453
1.0      25958
Name: result, dtype: int64


In [67]:
pro01 = np.array(x_test['fraud_ind_1'])
pro02 = np.array(x_test['pro'])
pro03 = np.array(x_test['stocn_pro'])
pro04 = np.array(x_test['csmcu_pro'])
pro05 = np.array(x_test['etymd_pro'])
pro06 = np.array(x_test['mcc_pro'])
pro07 = np.array(x_test['mchno_pro'])
pro08 = np.array(x_test['loctm_pro'])
pro09 = np.array(x_test['conam_pro'])

for i in range(len(pro01)):
    if pro02[i] >= 2.002471e-10:
        pro01[i]=1
    else:
        pro01[i]=0
#for i in range(len(pro01)):
#    if pro03[i] or pro04[i] or pro05[i] or pro06[i] or pro07[i] or pro08[i] or pro09[i] == 0:
#        pro01[i] =0
#    elif pro03[i] or pro04[i] or pro05[i] or pro06[i] or pro07[i] or pro08[i] or pro09[i] == 1:
#        pro01[i] = 1
        

In [54]:
y  = test['fraud_ind'].reset_index()

In [55]:
data_m['fraud_ind_1'] = data_m['fraud_ind_1'].fillna(0)

In [68]:
from sklearn.metrics import roc_auc_score
from sklearn import metrics
y_true = data_m['fraud_ind']
y_pred = pro1
auc = round(roc_auc_score(y_true, data_m['pro']),4)
f1_score = round(metrics.f1_score(y_true, y_pred),4)
accuracy = round(metrics.accuracy_score(y_true, y_pred),4)
precision = round(metrics.precision_score(y_true, y_pred),4)
recall = round(metrics.recall_score(y_true, y_pred),4)
print('AUC:', auc)
print('F1-score:', f1_score)
print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)

AUC: 0.9551
F1-score: 0.3762
Accuracy: 0.9773
Precision: 0.3125
Recall: 0.4725


In [ ]:
AUC: 0.9551
F1-score: 0.3762
Accuracy: 0.9773
Precision: 0.3125
Recall: 0.4725

In [71]:
y_true = y['fraud_ind']
y_pred = pro01
auc = round(roc_auc_score(y_true, x_test['pro']),4)
f1_score = round(metrics.f1_score(y_true, y_pred),4)
accuracy = round(metrics.accuracy_score(y_true, y_pred),4)
precision = round(metrics.precision_score(y_true, y_pred),4)
recall = round(metrics.recall_score(y_true, y_pred),4)
print('AUC:', auc)
print('F1-score:', f1_score)
print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)

AUC: 0.9261
F1-score: 0.248
Accuracy: 0.9782
Precision: 0.1837
Recall: 0.3816


In [ ]:
AUC: 0.9262
F1-score: 0.248
Accuracy: 0.9782
Precision: 0.1837
Recall: 0.3816